# Example 5: Generic cell signaling system in 3D

Geometry is divided into 4 domains; two volumes, and two surfaces:
- cytosol (Cyto): $\Omega_{Cyto}$
- endoplasmic reticulum volume (ER): $\Omega_{ER}$
- plasma membrane (PM): $\Gamma_{PM}$
- ER membrane (ERm): $\Gamma_{ERm}$

For simplicity, here we consider a "cube-within-a-cube" geometry, in which the smaller
inner cube represents a section of ER and one face of the outer cube ($x=0$) represents the PM. The other
faces of the outer cube are treated as no flux boundaries. The space outside
the inner cube but inside the outer cube is classified as cytosol.

There are three function-spaces on these three domains:

$$
u^{Cyto} = [A, B] \quad \text{on} \quad \Omega^{Cyto}\\
u^{ER} = [AER] \quad \text{on} \quad \Omega^{ER}\\
v^{ERm} = [R, Ro] \quad \text{on} \quad \Gamma^{ERm}
$$

In words, this says that species A and B reside in the cytosolic volume, 
species AER corresponds to an amount of species A that lives in the ER volume,
and species R (closed receptor/channel) and Ro (open receptor/channel) reside on the ER membrane.

In this model, species B reacts with a receptor/channel, R, on the ER membrane, causing it to open (change state from R->Ro), 
allowing species A to flow out of the ER and into the cytosol. 
Note that this is roughly similar to an IP<sub>3</sub> pulse at the PM, leading to Ca<sup>2+</sup> release from the ER,
where, by analogy, species B is similar to IP<sub>3</sub> and species A is similar to Ca<sup>2+</sup>. A more comprehensive
model of Ca<sup>2+</sup> dynamics in particular is implemented in Example 6.


## Code imports and initialization

In [ ]:
from matplotlib import pyplot as plt
import logging

import dolfin as d
import sympy as sym
import numpy as np
import pathlib
import gmsh  # must be imported before pyvista if dolfin is imported first

from smart import config, mesh, model, mesh_tools, visualization
from smart.model_assembly import (
    Compartment,
    Parameter,
    Reaction,
    Species,
    SpeciesContainer,
    ParameterContainer,
    CompartmentContainer,
    ReactionContainer,
)
from smart.units import unit
logger = logging.getLogger("smart")
logger.setLevel(logging.INFO)
# Aliases - base units
uM = unit.uM
um = unit.um
molecule = unit.molecule
sec = unit.sec
# Aliases - units used in model
D_unit = um**2 / sec
flux_unit = molecule / (um**2 * sec)
vol_unit = uM
surf_unit = molecule / um**2

## Define mesh and subdomain for restriction

In [ ]:
domain, facet_markers, cell_markers = mesh_tools.create_cubes()
visualization.plot_dolfin_mesh(domain, cell_markers, clip_plane=(1,
                               1, 0), clip_origin=(0.5, 0.5, 0.5))

restrict_mf = d.MeshFunction("size_t", domain, 2, 0)
for face in d.faces(domain):
    if face.midpoint().x() > d.DOLFIN_EPS and facet_markers[face] == 10:
        facet_markers[face] = 0
    elif face.midpoint().x() > 0.6 and facet_markers[face] == 12:
        restrict_mf[face] = 11
    elif face.midpoint().x() <= 0.6 and facet_markers[face] == 12:
        restrict_mf[face] = 12
mesh_folder = pathlib.Path("mesh")
mesh_folder.mkdir(exist_ok=True)
mesh_path = mesh_folder / "DemoCuboidsMesh.h5"

mesh_tools.write_mesh(
    domain, facet_markers, cell_markers, filename=mesh_path,
    subdomains = [restrict_mf],
)
parent_mesh = mesh.ParentMesh(
    mesh_filename=str(mesh_path),
    mesh_filetype="hdf5",
    name="parent_mesh",
    extra_keys=["subdomain0_2"]
)
restrict_mf = parent_mesh.subdomains[0]

## Define model and solve for different restriction cases

In [ ]:
restrict_sp = [True, False]
restrict_reactions = [False, True]
for i in range(len(restrict_sp)):
    Cyto = Compartment("Cyto", 3, um, 1)
    PM = Compartment("PM", 2, um, 10)
    ER = Compartment("ER", 3, um, 2)
    ERm = Compartment("ERm", 2, um, 12)
    PM.specify_nonadjacency(['ERm', 'ER'])
    ERm.specify_nonadjacency(['PM'])
    cc = CompartmentContainer()
    cc.add([ERm, ER, PM, Cyto])
    A = Species("A", 0.01, vol_unit, 1.0, D_unit, "Cyto")
    B = Species("B", 0.0, vol_unit, 1.0, D_unit, "Cyto")
    AER = Species("AER", 200.0, vol_unit, 5.0, D_unit, "ER")
    # Create an algebraic expression to define the initial condition of R
    Rinit = "(sin(40*y) + cos(40*z) + sin(40*x) + 3) * (y-x)**2"
    R1 = Species("R1", Rinit, surf_unit, 0.0, D_unit, "ERm")
    R1o = Species("R1o", 0.0, surf_unit, 0.0, D_unit, "ERm")
    sc = SpeciesContainer()
    sc.add([R1o, R1, AER, B, A])
    # Degradation of B in the cytosol
    k2f = Parameter("k2f", 10, 1 / sec)
    r2 = Reaction(
        "r2", ["B"], [], param_map={"on": "k2f"}, reaction_type="mass_action_forward"
    )

    # Activating receptors on ERm with B
    k3f = Parameter("k3f", 100, 1 / (uM * sec))
    k3r = Parameter("k3r", 100, 1 / sec)
    r3 = Reaction("r3", ["B", "R1"], ["R1o"], {"on": "k3f", "off": "k3r"})
    # Release of A from ERm to cytosol
    k4Vmax = Parameter("k4Vmax", 2000, 1 / (uM * sec))
    r4 = Reaction(
        "r4",
        ["AER"],
        ["A"],
        param_map={"Vmax": "k4Vmax"},
        species_map={"R1o": "R1o", "uER": "AER", "u": "A"},
        eqn_f_str="Vmax*R1o*(uER-u)",
    )

    if restrict_sp[i]: # restrict initial R1o distribution
        R1.restrict_to_subdomain(restrict_mf, 11)
    elif restrict_reactions[i]: # restrict both reactions involving R1o
        r3.restrict_to_subdomain(restrict_mf, 11)
        r4.restrict_to_subdomain(restrict_mf, 11)

    Vmax, t0, m = 500, 0.1, 200
    t = sym.symbols("t")
    pulseI = Vmax * sym.atan(m * (t - t0))
    pulse = sym.diff(pulseI, t)
    j1pulse = Parameter.from_expression(
        "j1pulse", pulse, flux_unit, use_preintegration=True, preint_sym_expr=pulseI
    )
    r1 = Reaction(
        "r1",
        [],
        ["B"],
        param_map={"J": "j1pulse"},
        eqn_f_str="J",
        explicit_restriction_to_domain="PM",
    )
    pc = ParameterContainer()
    pc.add([k4Vmax, k3r, k3f, k2f, j1pulse])
    rc = ReactionContainer()
    rc.add([r1, r2, r3, r4])

    conf = config.Config()
    conf.solver.update(
        {
            "final_t": 1,
            "initial_dt": 0.01,
            "time_precision": 6,
        }
    )
    model_cur = model.Model(pc, sc, cc, rc, conf, parent_mesh)
    model_cur.to_pickle('model_cur.pkl')
    model_cur.initialize()
    # Write initial condition(s) to file
    results = dict()
    result_folder = pathlib.Path(f"results{i}")
    result_folder.mkdir(exist_ok=True)
    for species_name, species in model_cur.sc.items:
        results[species_name] = d.XDMFFile(
            model_cur.mpi_comm_world, str(result_folder / f"{species_name}.xdmf")
        )
        results[species_name].parameters["flush_output"] = True
        results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)

    # Set loglevel to warning in order not to pollute notebook output
    logger.setLevel(logging.WARNING)

    avg_A = [A.initial_condition]
    # define integration measure and total volume for computing average A at each time point
    dx = d.Measure("dx", domain=model_cur.cc['Cyto'].dolfin_mesh)
    volume = d.assemble_mixed(1.0*dx)
    # Solve
    displayed = False
    while model_cur.t < model_cur.final_t:
        # Solve the system
        model_cur.monolithic_solve()
        # Save results for post processing
        for species_name, species in model_cur.sc.items:
            results[species_name].write(model_cur.sc[species_name].u["u"], model_cur.t)
        # compute average A concentration at each time step
        int_val = d.assemble_mixed(model_cur.sc['A'].u['u']*dx)
        avg_A.append(int_val / volume)

    plt.plot(model_cur.tvec, avg_A)
    
plt.xlabel('Time (s)')
plt.ylabel('Cytosolic concentration of A (μM)')